<a href="https://colab.research.google.com/github/kairamilanifitria/Bootcamp-NLP/blob/main/28082024_Deep_Learning_Model_Evaluation_%26_Improvement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization

In [ ]:
# Load and preprocess the data
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Define the neural network architecture
def create_model(learning_rate, num_hidden_layers, num_neurons):
    model = tf.keras.Sequential()
    for i in range(num_hidden_layers):
        model.add(tf.keras.layers.Dense(num_neurons, activation='relu'))
    model.add(tf.keras.layers.Dense(2, activation='softmax'))
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
# Define the objective function to optimize
def objective(learning_rate, num_hidden_layers, num_neurons):
    model = create_model(learning_rate, int(num_hidden_layers), int(num_neurons))
    model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=0)
    val_loss, val_acc = model.evaluate(X_test, y_test, verbose=0)
    return val_acc

In [ ]:
# Define the search space for hyperparameters
pbounds = {'learning_rate': (0.0001, 0.1),
           'num_hidden_layers': (1, 5),
           'num_neurons': (5, 50)}

In [ ]:
# Perform Bayesian optimization
optimizer = BayesianOptimization(f=objective, pbounds=pbounds, verbose=2)
optimizer.maximize(init_points=5, n_iter=20)

|   iter    |  target   | learni... | num_hi... | num_ne... |
-------------------------------------------------------------
| 1         | 0.9649    | 0.03881   | 4.927     | 48.78     |
| 2         | 0.9649    | 0.02806   | 1.698     | 29.59     |
| 3         | 0.9561    | 0.07309   | 3.696     | 49.21     |
| 4         | 0.9737    | 0.0891    | 4.311     | 15.79     |
| 5         | 0.9825    | 0.01807   | 1.601     | 37.91     |
| 6         | 0.9737    | 0.00625   | 1.245     | 39.17     |
| 7         | 0.9825    | 0.0006947 | 2.771     | 37.32     |
| 8         | 0.9649    | 0.0598    | 1.094     | 35.88     |
| 9         | 0.9649    | 0.1       | 3.033     | 38.57     |
| 10        | 0.7895    | 0.0001    | 2.01      | 37.2      |
| 11        | 0.9649    | 0.05827   | 2.264     | 42.86     |
| 12        | 0.9825    | 0.09976   | 2.124     | 36.52     |
| 13        | 0.9561    | 0.09731   | 4.982     | 18.94     |
| 14        | 0.9825    | 0.0568    | 1.793     | 36.89     |
| 15    

In [ ]:
# Print the optimized hyperparameters and validation accuracy
print('Optimized hyperparameters:')
print(optimizer.max['params'])
print('Validation accuracy: {:.2f}%'.format(optimizer.max['target'] * 100))

Optimized hyperparameters:
{'learning_rate': 0.0821362245416636, 'num_hidden_layers': 4.990434414597773, 'num_neurons': 20.70197904551883}
Validation accuracy: 99.12%
